In [28]:
import pandas as pd
from string import punctuation
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/marcus/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

### Load dataset

In [25]:
df = pd.read_pickle('data/extracted.pickle')

### Process Natural Language


In [40]:
stop_words = set(stopwords.words('english'))

def preprocess_docstrings(docstring):
    docstring = docstring.lower()
    docstring = docstring.translate(str.maketrans(' ', ' ', punctuation))
    docstring = ' '.join([w for w in docstring.split() if w.lower() not in stop_words])

    return docstring

In [41]:
df.docstring = df.docstring.apply(preprocess_docstrings)
df.docstring.head()

0    attempt convert specified value epoch time git...
1    generate server side cookie given cookie value...
2    generate httponly cookie hs2 cookie param cook...
3    copies files source target sets executable fla...
4                        saves configuration info disk
Name: docstring, dtype: object

### Process Code Strings

### Save preprocessed dataset

In [42]:
df.to_pickle('data/processed.pickle')